In [12]:
import json
import pymysql
import os

In [13]:
# Database Setup
# --------------------------------------------------
def setupdb():
    f = open('../application/config/localdb_config.json')
    dbconfig = json.loads(f.read())
    db = pymysql.connect(host=dbconfig['localhost'], user=dbconfig['username'], password=dbconfig['password'], database=dbconfig['database'])
    f.close()
    return db

## User Class
---

In [14]:
class User:
    def __init__(self, userid, fname, lname, email, ski=0, snowboard=0, stance=None, permissions=[], region=None, photo=None):
        self.id = userid
        self.fname = fname
        self.lname = lname
        self.email = email
        self.ski = ski
        self.snowboard = snowboard
        self.stance = stance
        self.permissions = permissions
        self.region = region
        self.photo = photo


    # --------------------------------------------------
    # Create New User                    F U N C T I O N
    # --------------------------------------------------
    def save(self):
        db = setupdb()
        cursor = db.cursor()
        try:
            sql = """insert into 'Users' (user_id, fname, lname, email, ski, snowboard, stance, region, permissions, created, updated, photo)
                values ({}, {}, {}, {}, {}, {}, {}, {}))
            """.format(self.id, self.fname, self.lname, self.email, 
                       self.ski, self.snowboard, self.stance, self.region, '~'.join(self.permissions), 
                       datetime.now(pytz.timezone('Canada/Pacific')), datetime.now(pytz.timezone('Canada/Pacific')),
                       self.photo)
            cursor.execute(sql)
            db.commit()
            #self.id = cursor.execute("SELECT last_insert_rowid() FROM songs").fetchone()[0]
             
        except Exception as e:
            print("Could not create new user:\n{}".format(e))   
            return False

        print("Created New User:\n{} {} ~ {}\nPermissions: {}\nSki: {} Snowboard: ({})\n{}Region: {}"
                     .format(self.fname, self.lname, self.email, ', '.join(self.permissions), self.ski, self.snowboard, self.stance, self.region))

        return True
    

    # ToDo...
    # Return User object
    # --------------------------------------------------
    # Get User                           F U N C T I O N
    # --------------------------------------------------
    @classmethod
    def get(cls, id=None, email=None):

        db = setupdb()
        cursor = db.cursor()

        # Get user by ID
        if id:
            try:
                print("Getting user from ID: {}".format(id))
                sql = """SELECT * FROM Users WHERE user_id = '{}'""".format(id)
                cursor.execute(sql)
                result = cursor.fetchone()
                print("Result: {}".format(result))
            
            except Exception as e:
                print(e)

        # Get user by email
        elif email:
            try:
                print("Getting user from EMAIL: {}".format(email))
                sql = """SELECT * FROM Users WHERE email = '{}'""".format(email)
                cursor.execute(sql)
                result = cursor.fetchone()
                print("Result: {}".format(result))

            except Exception as e:
                print(e)

        if not result:
            print("EMPTY")
            return None
                
        # Map DB Result to User Object
        user = User(
            userid=result[0], 
            fname=result[1], 
            lname=result[2], 
            email=result[3], 
            ski=result[4], 
            snowboard=result[5],
            stance=result[6],
            region=result[7],
            permissions=result[8],
            photo=result[9],
        )
        
        if user.permissions:
            user.permissions = user.permissions.split('~')

        return user

## User Tests
---

In [15]:
liam = User.get(email='lck3nny@gmail.com')

Getting user from EMAIL: lck3nny@gmail.com
Result: ('GOn89KKWumTycl3uJ3lZwProK5j1', 'Liam', 'Kenny', 'lck3nny@gmail.com', None, None, None, None, None, datetime.datetime(2023, 12, 19, 16, 17, 46), datetime.datetime(2023, 12, 19, 16, 17, 46), None)


In [16]:
liam.__dict__

{'id': 'GOn89KKWumTycl3uJ3lZwProK5j1',
 'fname': 'Liam',
 'lname': 'Kenny',
 'email': 'lck3nny@gmail.com',
 'ski': None,
 'snowboard': None,
 'stance': None,
 'permissions': None,
 'region': None,
 'photo': datetime.datetime(2023, 12, 19, 16, 17, 46)}

In [17]:
import json
import pymysql
import os
import sys
import os.path
import logging
import json
import pytz
from datetime import datetime
from operator import itemgetter
from difflib import SequenceMatcher
from elasticsearch import Elasticsearch
from elasticsearch.client import IndicesClient


In [18]:
# Connect ElasticSearch credentials
f = open(sys.path[0] + '/../application/config/bonsai_config.json')
es_config = json.load(f)

In [19]:
es_endpoint = es_config['url']
es_user = es_config['key']
es_password = es_config['secret']

In [20]:
es_client = Elasticsearch([es_endpoint], basic_auth=(es_user, es_password))
idx_manager = IndicesClient(es_client)

In [21]:
# Database Setup
# --------------------------------------------------
def setupdb():
    f = open('../application/config/localdb_config.json')
    dbconfig = json.loads(f.read())
    db = pymysql.connect(host=dbconfig['localhost'], user=dbconfig['username'], password=dbconfig['password'], database=dbconfig['database'])
    f.close()
    return db

## SkiBoard Class
---

In [111]:
class SkiBoard():

    # If a SkiBoard has an ID of 0 it has not been saved in the database
    def __init__(self, skiboard_id, brand, model, year, name, category, description=None, stiffness=None, family=None, flex_profile=None, camber_profile=None, camber_details=[], core=None, laminates=[], base=None, sidewall=None, weight=0, youth=False, url=None):
        self.id = skiboard_id
        self.brand = brand
        self.model = model
        self.year = year
        self.name = name
        self.category = category
        self.description = description
        self.stiffness = stiffness
        self.family = family
        self.flex_profile = flex_profile
        self.camber_profile = camber_profile
        self.camber_details = camber_details
        self.core = core
        self.laminates = laminates
        self.base = base
        self.sidewall = sidewall
        self.weight = weight
        self.youth = youth
        self.url = url

    # --------------------------------------------------
    # Is Duplicate                       F U N C T I O N
    # --------------------------------------------------
    def is_duplicate(self):
        db = setupdb()
        cursor = db.cursor()

        # Search for SkiBoards with 
        try:
            logging.info("Checking for Duplicate SkiBoards: {} {} ({})".format(self.brand, self.model, self.year))
            sql = """SELECT skiboard_id FROM SkiBoards WHERE brand = '{}' AND model = '{}' AND year = '{}'""".format(self.brand, self.model, self.year)
            cursor.execute(sql)
            result = cursor.fetchone()
            logging.info("Duplicate Found: {}".format(result))
        except Exception as e:
            logging.error(e)

        if result:
            return True

        return False

    # --------------------------------------------------
    # Get Item                           F U N C T I O N
    # --------------------------------------------------
    @classmethod
    def get(cls, id=None, brand=None, model=None, year=None):
        
        db = setupdb()
        cursor = db.cursor()

        if id:
            try:
                logging.info("Getting SkiBoard from ID: {}".format(id))
                sql = """SELECT * FROM SkiBoards WHERE skiboard_id = '{}'""".format(id)
                cursor.execute(sql)
                result = cursor.fetchone()
                logging.info("Result: {}".format(result))
            except Exception as e:
                logging.error(e)
                return None
            
        elif brand and model and year:
            try:
                logging.info("Getting SkiBoard by B-M-Y: {} {} ({})".format(brand, model, year))
                sql = """SELECT * FROM SkiBoards WHERE brand = '{}' AND model = '{}' AND year = '{}'""".format(brand, model, year)
                cursor.execute(sql)
                result = cursor.fetchone()
                logging.info("Result: {}".format(result))
            except Exception as e:
                logging.error(e)
                return None
            
        if not result:
            return None
        
        # Map DB Result to User Object
        skiboard = SkiBoard(
            skiboard_id=result[0], 
            url=result[1], 
            brand=result[2], 
            model=result[3], 
            year=result[4], 
            category=result[5],
            family=result[6],
            description=result[7],
            stiffness=result[8],
            flex_profile=result[9],
            camber_profile=result[10],
            camber_details=result[11],
            core=result[12],
            laminates=result[13],
            base=result[14],
            sidewall=result[15],
            weight=result[16],
            youth=result[18]
        )
        
        return skiboard


    # --------------------------------------------------
    # Save SkiBoard                      F U N C T I O N
    # --------------------------------------------------
    def save(self):
        db = setupdb()
        cursor = db.cursor()

        try:
            if self.id:
                print("Updating SkiBoard...")
                sql = f"""REPLACE INTO SkiBoards (skiboard_id, url, brand, model, year, name, category, family, description, stiffness, flex_profile, camber_profile, camber_details, core, laminates, base, sidewall, weight, youth, updated) 
                values(
                '{str(self.id)}'
                '{str(self.url)}', 
                '{str(self.brand)}', 
                '{str(self.model)}', 
                '{str(self.year)}',
                '{str(self.brand)} {str(self.model)} {str(self.year)}',
                '{str(self.category)}', 
                '{str(self.family)}', 
                '{str(self.description)}', 
                {float(self.stiffness)}, 
                '{str(self.flex_profile)}', 
                '{str(self.camber_profile)}', 
                '{'~'.join(str(i) for i in self.camber_details)}', 
                '{str(self.core)}', 
                '{'~'.join(str(i) for i in self.laminates)}', 
                '{str(self.base)}', 
                '{str(self.sidewall)}', 
                {float(self.weight)}, 
                {bool(self.youth)}, 
                '{datetime.now(pytz.timezone('Canada/Pacific')).strftime("%Y/%m/%d %H:%M:%S")}' )"""
                
            else:
                print("Creating SkiBoard...")
                sql = f"""INSERT INTO SkiBoards (url, brand, model, year, name, category, family, description, stiffness, flex_profile, camber_profile, camber_details, core, laminates, base, sidewall, weight, youth, created, updated) 
                values(
                '{str(self.url)}', 
                '{str(self.brand)}', 
                '{str(self.model)}', 
                '{str(self.year)}', 
                '{str(self.brand)} {str(self.model)} {str(self.year)}',
                '{str(self.category)}', 
                '{str(self.family)}', 
                '{str(self.description)}', 
                {float(self.stiffness)}, 
                '{str(self.flex_profile)}', 
                '{str(self.camber_profile)}', 
                '{'~'.join(str(i) for i in self.camber_details)}', 
                '{str(self.core)}', 
                '{'~'.join(str(i) for i in self.laminates)}', 
                '{str(self.base)}', 
                '{str(self.sidewall)}', 
                {float(self.weight)}, 
                {int(bool(self.youth))}, 
                '{datetime.now(pytz.timezone('Canada/Pacific')).strftime("%Y/%m/%d %H:%M:%S")}',
                '{datetime.now(pytz.timezone('Canada/Pacific')).strftime("%Y/%m/%d %H:%M:%S")}')"""
                
            
            print(f"About to execute SQL: {sql}")
            cursor.execute(sql)
            db.commit()
             
        except Exception as e:
            logging.error("Could not create new SkiBoard:\n{}".format(e))
            print("Could not create new SkiBoard: {}".format(e))
            return False

        logging.info("Saved SkiBoard:\nBrand: {}\nModel: {} Year: ({})".format(self.brand, self.model, self.year))

        # ToDo...
        # Update ElasticSearch
        '''
        successes = 0
        logging.info("Uploading SkiBoard to ElasticSearch")
        es.update(
            id=self.id,
            index='SkiBoards',
            document=self.__dict__
        )   
        '''
        

        return True
    
    # --------------------------------------------------
    # Search Database                    F U N C T I O N
    # -------------------------------------------------- 
    @classmethod
    def search_db(cls, query_string):
        
        db = setupdb()
        cursor = db.cursor()

        try:
            #logging.info("Searching for SkiBoard: {}".format(query_string))
            print(f"Searching for SkiBoard: {query_string}")
            sql = f"SELECT * FROM SkiBoards WHERE MATCH(name) AGAINST('{query_string}' IN NATURAL LANGUAGE MODE)"
            # sql = f"SELECT * FROM SkiBoards WHERE MATCH(name) AGAINST('{query_string}' WITH QUERY EXPANSION)"
            cursor.execute(sql)
            response = cursor.fetchall()
            logging.info("Response: {}".format(response))
            print(f"Response: {response}")
        except Exception as e:
            logging.error(e)
            return None
        
        results = []
        for r in response:
            # Map DB Result to User Object
            result = SkiBoard(
                skiboard_id=r[0], 
                url=r[1], 
                brand=r[2], 
                model=r[3], 
                year=r[4], 
                name=r[5],
                category=r[6],
                family=r[7],
                description=r[8],
                stiffness=r[9],
                flex_profile=r[10],
                camber_profile=r[11],
                camber_details=r[12],
                core=r[13],
                laminates=r[14],
                base=r[15],
                sidewall=r[16],
                weight=r[17],
                youth=r[18]
            )
            try:
                print(f"Getting sizes of skibaord: {result[0]}")
                sql = f"SELECT * FROM Sizes WHERE skiboard_id = {result[0]}"
                cursor.execute(sql)
                response = cursor.fetchall()
                print(f"Size results: {response}")
            except Exception as e:
                logging.error("Unable to retreive sizes for skiboard")

        return results
    


In [101]:
burton_custom = SkiBoard(
    skiboard_id=0, 
    brand='Burton', 
    model='Custom', 
    year=2020, 
    category='Snowboard', 
    description='''The most trusted board ever'''.replace("'", "`"),
    stiffness=6, 
    family=None, 
    flex_profile='Twin', 
    camber_profile='Full Camber', 
    camber_details=[], 
    core='Super Fly II™ 700G', 
    laminates=['45° Carbon Highlights'], 
    base='Sintered WFO Base',
    sidewall=None, 
    weight=0,
    youth=False, 
    url=None)

In [102]:
for key in vars(burton_custom):
    print(f'{key}: {vars(burton_custom)[key]}')

id: 0
brand: Burton
model: Custom
year: 2020
category: Snowboard
description: The most trusted board ever
stiffness: 6
family: None
flex_profile: Twin
camber_profile: Full Camber
camber_details: []
core: Super Fly II™ 700G
laminates: ['45° Carbon Highlights']
base: Sintered WFO Base
sidewall: None
weight: 0
youth: False
url: None


In [103]:
burton_custom.save()

Creating SkiBoard...
About to execute SQL: INSERT INTO SkiBoards (url, brand, model, year, name, category, family, description, stiffness, flex_profile, camber_profile, camber_details, core, laminates, base, sidewall, weight, youth, created, updated) 
                values(
                'None', 
                'Burton', 
                'Custom', 
                '2020', 
                'Burton Custom 2020',
                'Snowboard', 
                'None', 
                'The most trusted board ever', 
                6.0, 
                'Twin', 
                'Full Camber', 
                '', 
                'Super Fly II™ 700G', 
                '45° Carbon Highlights', 
                'Sintered WFO Base', 
                'None', 
                0.0, 
                0, 
                '2024/01/27 10:33:32',
                '2024/01/27 10:33:32')


True

In [112]:
results = SkiBoard.search_db("Burton")
if results:
    print(results)
else:
    print("No Results")

ERROR:root:Unable to retreive sizes for skiboard


Searching for SkiBoard: Burton
Response: ((1, '', 'Burton', 'Custom', '2020', 'Burton Custom 2020', 'Snowboard', '', None, 6.0, 'Twin', 'Full Camber', None, 'Super Fly II™ 700G Core', '45° Carbon Highlights', 'Sintered WFO Base', None, None, 0, datetime.datetime(2024, 1, 22, 12, 8, 15), datetime.datetime(2024, 1, 22, 12, 8, 15)),)
No Results


In [105]:
query_string = 'Burton Process'

In [88]:
db = setupdb()
cursor = db.cursor()
sql = f"SELECT * FROM SkiBoards WHERE MATCH(name) AGAINST('{query_string}' IN NATURAL LANGUAGE MODE)"
sql = f"SELECT * FROM SkiBoards WHERE MATCH(name) AGAINST('{query_string}' WITH QUERY EXPANSION)"
cursor.execute(sql)
results = cursor.fetchall()

In [89]:
print(results)


(('AUTO_INCREMENT', '', 'Burton', 'Custom', '2020', 'Burton Custom 2020', 'Snowboard', '', None, 6.0, 'Twin', 'Full Camber', None, 'Super Fly II™ 700G Core', '45° Carbon Highlights', 'Sintered WFO Base', None, None, 0, datetime.datetime(2024, 1, 22, 12, 8, 15), datetime.datetime(2024, 1, 22, 12, 8, 15)),)


In [93]:
db = setupdb()
cursor = db.cursor()
sql = """INSERT INTO SkiBoards (url, brand, model, year, name, category, family, description, stiffness, flex_profile, camber_profile, camber_details, core, laminates, base, sidewall, weight, youth, created, updated) 
                values(
                'None', 
                'Burton', 
                'Custom', 
                '2020', 
                'Burton Custom 2020',
                'Snowboard', 
                'None', 
                'The most trusted board ever', 
                6.0, 
                'Twin', 
                'Full Camber', 
                '', 
                'Super Fly II™ 700G', 
                '45° Carbon Highlights', 
                'Sintered WFO Base', 
                'None', 
                0.0, 
                0, 
                '2024/01/22 13:30:05',
                '2024/01/22 13:30:05')"""
cursor.execute(sql)
db.commit()